# Rcpp

Rcpp makes it easy to call C++ functions from R
* Users doesn't have to worry about
* platform;
* compilers;
* R/C interface.

## Example 1

In [1]:
# R implementation

f <- function(n) {
    if (n < 2) return(n)
    return(f(n-1) + f(n-2))
}

In [2]:
sapply(0:10, f)

[1]  0  1  1  2  3  5  8 13 21 34 55

In [3]:
# R C++ implementation

Rcpp::cppFunction("
    int g(int n) {
        if (n < 2) return(n);
        return(g(n-1) + g(n-2));
    }
")

In [4]:
sapply(0:10, g)

[1]  0  1  1  2  3  5  8 13 21 34 55

In [6]:
rbenchmark::benchmark(f(25), g(25), order = 'relative')[,1:4]

,test,replications,elapsed,relative
2,g(25),100,0.039,1.000
1,f(25),100,8.614,220.872


## C/C++ data types

Type | Description

* `char` | A single character.
* `int` | An integer.
* `float` | A single precision floating point number.
* `double` | A double-precision floating point number.
* `void` | A valueless quantity.
* (auto) ?


## The sourceCpp function

* The `cppFunction` is great for small examples
* But you should put C++ code in a separate file (`.cpp`)
* `sourceCpp("path/to/file.cpp")`
* The `.cpp` needs a few headers

## Namespace

* To access the Rcpp functions we would have to type `Rcpp::function_1`
* To avoid typing `Rcpp::`
```{r engine="Rcpp", eval=FALSE}
using namespace Rcpp;
```
* Above each function we want to export/use in R
```{r engine="Rcpp", eval=FALSE}
// [[Rcpp::export]]
```

## Complete file

```{r engine="Rcpp"}
# include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
double add_c(double x, double y) {
  double value = x + y;
  return value;
}

## Example 2

In [7]:
mean_r = function(x) {
  n = length(x)
  m = 0
  for(i in seq_along(x))
    m = m + x[i]/n
  m
}

In [8]:
Rcpp::cppFunction('
    double mean_c(NumericVector x){
        int i;
        int n = x.size();
        double mean = 0;

        for(i=0; i<n; i++) {
            mean = mean + x[i]/n;
        }
        return mean;
    }'
)

In [10]:
x = rnorm(1e4)

z = microbenchmark::microbenchmark(
    mean(x),
    mean_r(x),
    mean_c(x)
)
print(z)

Unit: microseconds
      expr     min       lq      mean   median       uq      max neval
   mean(x)  18.411  18.8195  21.77282  19.0585  20.1885  119.299   100
 mean_r(x) 433.510 435.1070 502.23063 455.1160 498.8890 2911.686   100
 mean_c(x)  38.811  39.2050  47.71543  39.5300  43.9205  547.767   100
